# Wikipedia Retriever

In [1]:
from langchain_community.retrievers import WikipediaRetriever

In [2]:
retriever  = WikipediaRetriever(top_k_results=2, lang="en")

In [3]:
query = "the geopolitical history of india and pakistan from the perspective of a chinese"

In [4]:
docs = retriever.invoke(query)

In [5]:
docs

[Document(metadata={'title': 'India–Pakistan war of 1965', 'summary': "The India–Pakistan war of 1965, also known as the second India–Pakistan war, was an armed conflict between Pakistan and India that took place from August 1965 to September 1965.\nThe conflict began following Pakistan's unsuccessful Operation Gibraltar, which was designed to infiltrate forces into Jammu and Kashmir to precipitate an insurgency against Indian rule. The seventeen day war caused thousands of casualties on both sides and witnessed the largest engagement of armoured vehicles and the largest tank battle since World War II. Hostilities between the two countries ended after a ceasefire was declared through UNSC Resolution 211 following a diplomatic intervention by the Soviet Union and the United States, and the subsequent issuance of the Tashkent Declaration. Much of the war was fought by the countries' land forces in Kashmir and along the border between India and Pakistan. This war saw the largest amassing 

In [6]:
for i, doc in enumerate(docs):
    print(f"\n--- Result {i+1}---")
    print(f"Content:\n{doc.page_content}...")


--- Result 1---
Content:
The India–Pakistan war of 1965, also known as the second India–Pakistan war, was an armed conflict between Pakistan and India that took place from August 1965 to September 1965.
The conflict began following Pakistan's unsuccessful Operation Gibraltar, which was designed to infiltrate forces into Jammu and Kashmir to precipitate an insurgency against Indian rule. The seventeen day war caused thousands of casualties on both sides and witnessed the largest engagement of armoured vehicles and the largest tank battle since World War II. Hostilities between the two countries ended after a ceasefire was declared through UNSC Resolution 211 following a diplomatic intervention by the Soviet Union and the United States, and the subsequent issuance of the Tashkent Declaration. Much of the war was fought by the countries' land forces in Kashmir and along the border between India and Pakistan. This war saw the largest amassing of troops in Kashmir since the Partition of In

# Vector Store Retriever

In [7]:
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document

d:\14_GenAI\myvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [9]:
embeddings = OllamaEmbeddings(model="llama3")

#Create Chroma vector store in memory
vectorize = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="collection"
)

In [10]:
#Convert vectorstore into a retriever
retriever = vectorize.as_retriever(
    search_kwargs={"k":2}
)

In [11]:
query = "What is Chroma used for?"
result = retriever.invoke(query)

In [12]:
for i, doc in enumerate(result):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
LangChain helps developers build LLM applications easily.


# MMR Retriever

In [13]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [14]:
from langchain_community.vectorstores import FAISS

embeddings = OllamaEmbeddings(model="llama3")

vectorStore = FAISS.from_documents(
    documents=docs,
    embedding=embeddings
)

In [15]:
retriever = vectorStore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "lambda_mult": 0.5}  #k = top results, lambda_mult = relevance-diversity balance
)

In [16]:
query = "What is LangChain?"
results = retriever.invoke(query)

In [17]:
for i, doc in enumerate(results):
    print(f"--- Results {i+1} ---")
    print(doc.page_content)

--- Results 1 ---
LangChain is used to build LLM based applications.
--- Results 2 ---
Embeddings are vector representations of text.
--- Results 3 ---
MMR helps you get diverse results when doing similarity search.


# ContextualCompressionRetriever

In [18]:
from langchain_classic.retrievers import ContextualCompressionRetriever

In [19]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [20]:
embeddings = OllamaEmbeddings(model="llama3")

vectorStore = FAISS.from_documents(
    documents=docs,
    embedding=embeddings
)

In [21]:
base_retriever = vectorStore.as_retriever(search_kwargs={"k": 5})

In [22]:
from langchain_classic.retrievers.document_compressors import LLMChainExtractor
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3")

In [23]:
compressor = LLMChainExtractor.from_llm(llm)

In [24]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [25]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [26]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
The extracted relevant part is:

>The chlorophyll in plant cells captures sunlight during photosynthesis.

This part of the context directly answers the question "What is photosynthesis?"

--- Result 3 ---
NO_OUTPUT. None of the context is relevant to answer the question about photosynthesis.


# Multiquery Retriever

In [27]:
from langchain_classic.retrievers import MultiQueryRetriever

In [28]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [29]:
embeddings = OllamaEmbeddings(model="llama3")

In [30]:
vectorStore = FAISS.from_documents(documents=all_docs, embedding=embeddings)

In [31]:
similarity_retriever = vectorStore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [32]:
mutltiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorStore.as_retriever(search_kwargs={"k": 5}),
    llm=ChatOllama(model="llama3")
)

In [33]:
query = "How to improve energy levels and maintain balance?"

In [34]:
# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= mutltiquery_retriever.invoke(query)

In [35]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 2 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 3 ---
Deep sleep is crucial for cellular repair and emotional regulation.

--- Result 4 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 5 ---
Regular walking boosts heart health and can reduce symptoms of depression.
******************************************************************************************************************************************************

--- Result 1 ---
Python balances readability with power, making it a popular system design language.

--- Result 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 3 ---
Photosynthesis enables plants to produce energy by converting sunlight.

--- Result 4 ---
Regular walking boosts heart health and can reduce symptoms of depr